# 🌤️ PV Power Estimation - Interactive Demo

**Multi-Modal Deep Learning for Solar Power Prediction**

This notebook demonstrates the PV Power Estimation system that predicts DC power output using:
- 📸 Sky images (cloud cover, opacity)
- 🌡️ Weather sensor data
- ☀️ Sun position calculations

---

## 1️⃣ Setup & Installation

Run the cell below to install all dependencies:

In [ ]:
# Clone the repository
!git clone https://github.com/Pradsey5010/PV-Power-Estimation.git
%cd PV-Power-Estimation/sky_power_estimation

# Install dependencies
!pip install -q torch torchvision timm numpy pandas pillow plotly streamlit scikit-learn pyyaml tqdm

print("✅ Setup complete!")

## 2️⃣ Import Libraries

In [ ]:
import sys
sys.path.insert(0, '.')

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Import project modules
from sky_power_estimation.models.sky_power_model import SkyPowerModel
from sky_power_estimation.models.image_encoder import ImageEncoder
from sky_power_estimation.models.temporal_encoder import TemporalEncoder
from sky_power_estimation.utils.sun_position import SunPositionCalculator
from sky_power_estimation.utils.weather_processor import WeatherProcessor
from sky_power_estimation.utils.image_processor import ImageProcessor, generate_synthetic_sky_image

print("✅ All imports successful!")
print(f"📦 PyTorch version: {torch.__version__}")
print(f"🖥️ Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 3️⃣ Create the Model

Let's create a multi-modal model with:
- ResNet backbone for sky images
- LSTM temporal encoder
- Attention-based fusion

In [ ]:
# Create the model
model = SkyPowerModel(
    image_backbone="resnet18",     # CNN for sky images
    image_pretrained=True,          # Use ImageNet pretrained weights
    image_feature_dim=256,          # Image feature dimension
    temporal_type="lstm",           # LSTM for temporal patterns
    temporal_hidden_dim=128,        # Temporal hidden size
    fusion_method="attention",      # Attention-based fusion
    fusion_dim=256,                 # Fusion output dimension
    output_hidden_dims=[128, 64]    # Output MLP layers
)

# Print model summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"🧠 Model created successfully!")
print(f"📊 Total parameters: {total_params:,}")
print(f"🎯 Trainable parameters: {trainable_params:,}")

## 4️⃣ Generate Synthetic Sky Images

Let's create some synthetic sky images with different cloud covers:

In [ ]:
# Generate sky images with different cloud covers
cloud_covers = [0.1, 0.3, 0.5, 0.7, 0.9]

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for i, cloud_cover in enumerate(cloud_covers):
    sky_image = generate_synthetic_sky_image(400, 300, cloud_cover)
    axes[i].imshow(sky_image)
    axes[i].set_title(f"Cloud Cover: {int(cloud_cover*100)}%")
    axes[i].axis('off')

plt.suptitle("Synthetic Sky Images with Varying Cloud Cover", fontsize=14)
plt.tight_layout()
plt.show()

## 5️⃣ Cloud Feature Extraction

Extract cloud features from sky images:

In [ ]:
# Initialize image processor
image_processor = ImageProcessor(image_size=(224, 224))

# Generate a sample image
sample_image = generate_synthetic_sky_image(640, 480, cloud_cover=0.4)

# Extract cloud features
features = image_processor.extract_cloud_features(sample_image)

print("☁️ Cloud Features Extracted:")
print("-" * 40)
for key, value in features.items():
    print(f"  {key}: {value:.3f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].imshow(sample_image)
axes[0].set_title("Original Sky Image")
axes[0].axis('off')

# Cloud segmentation
cloud_mask = image_processor.segment_clouds(sample_image)
axes[1].imshow(cloud_mask, cmap='gray')
axes[1].set_title(f"Cloud Mask (Cover: {features['cloud_cover']:.1f}%)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 6️⃣ Sun Position Calculation

Calculate sun position using astronomical algorithms:

In [ ]:
# Initialize sun position calculator for San Francisco
sun_calc = SunPositionCalculator(
    latitude=37.7749,
    longitude=-122.4194,
    timezone="US/Pacific"
)

# Get sun position for current time
current_time = datetime.now()
sun_pos = sun_calc.get_sun_position(current_time)

print("☀️ Sun Position:")
print("-" * 40)
print(f"  Time: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"  Zenith: {sun_pos['zenith']:.2f}°")
print(f"  Azimuth: {sun_pos['azimuth']:.2f}°")
print(f"  Elevation: {sun_pos['apparent_elevation']:.2f}°")
print(f"  Daytime: {'Yes ☀️' if sun_calc.is_daytime(current_time) else 'No 🌙'}")

# Plot sun path for the day
hours = np.arange(0, 24, 0.5)
elevations = []
azimuths = []

for h in hours:
    test_time = current_time.replace(hour=int(h), minute=int((h % 1) * 60))
    pos = sun_calc.get_sun_position(test_time)
    elevations.append(pos['apparent_elevation'])
    azimuths.append(pos['azimuth'])

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(hours, elevations, 'b-', linewidth=2)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.fill_between(hours, elevations, 0, where=np.array(elevations) > 0, alpha=0.3)
plt.xlabel('Hour of Day')
plt.ylabel('Elevation (°)')
plt.title('Sun Elevation Throughout the Day')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(azimuths, elevations, 'orange', linewidth=2)
plt.xlabel('Azimuth (°)')
plt.ylabel('Elevation (°)')
plt.title('Sun Path (Azimuth vs Elevation)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7️⃣ Run Prediction

Run a power prediction with the model:

In [ ]:
# Set model to evaluation mode
model.eval()

# Prepare inputs
batch_size = 1

# Generate and preprocess sky image
sky_image = generate_synthetic_sky_image(640, 480, cloud_cover=0.3)
processed_image = image_processor.preprocess(sky_image)
image_tensor = torch.from_numpy(processed_image).unsqueeze(0)  # [1, 3, 224, 224]

# Weather data (normalized)
weather_data = torch.tensor([[
    0.5,   # temperature (normalized)
    0.6,   # humidity
    0.5,   # pressure
    0.2,   # wind_speed
    0.5,   # wind_direction
    0.4,   # dew_point
    0.7,   # ghi
    0.6,   # dni
    0.3    # dhi
]], dtype=torch.float32)

# Sun position features (normalized)
sun_features = sun_calc.get_features_array(datetime.now())
sun_tensor = torch.tensor([[
    sun_features[0] / 180,  # zenith
    sun_features[1] / 360,  # azimuth
    (sun_features[2] + 90) / 180,  # elevation
    0.5  # equation of time
]], dtype=torch.float32)

# Run prediction
with torch.no_grad():
    prediction = model(
        current_image=image_tensor,
        current_weather=weather_data,
        current_sun_position=sun_tensor
    )

# Scale prediction (model outputs are not trained, so we simulate)
# In a real scenario, the model would be trained on actual power data
simulated_power = abs(prediction.item()) * 500 + 200

print("⚡ Power Prediction Result:")
print("-" * 40)
print(f"  Predicted DC Power: {simulated_power:.1f} W")
print(f"  Cloud Cover: ~30%")
print(f"  Sun Elevation: {sun_features[2]:.1f}°")

## 8️⃣ Model Architecture Visualization

In [ ]:
print("\n🏗️ MODEL ARCHITECTURE")
print("=" * 60)
print("""
┌─────────────────────────────────────────────────────────┐
│                   SKY POWER MODEL                       │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐     │
│  │  Sky Image  │  │   Weather   │  │ Sun Position│     │
│  │  224x224x3  │  │  9 features │  │  4 features │     │
│  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘     │
│         │                │                │            │
│         ▼                └────────┬───────┘            │
│  ┌──────────────┐                 ▼                    │
│  │ ResNet-18    │       ┌──────────────────┐           │
│  │ (Pretrained) │       │  Weather Encoder │           │
│  └──────┬───────┘       │    (MLP x 2)     │           │
│         │               └────────┬─────────┘           │
│         │                        │                     │
│         └──────────┬─────────────┘                     │
│                    ▼                                   │
│          ┌──────────────────┐                          │
│          │  LSTM Encoder    │                          │
│          │  (Bidirectional) │                          │
│          └────────┬─────────┘                          │
│                   ▼                                    │
│          ┌──────────────────┐                          │
│          │ Attention Fusion │                          │
│          │  (Cross-Modal)   │                          │
│          └────────┬─────────┘                          │
│                   ▼                                    │
│          ┌──────────────────┐                          │
│          │   Output Head    │                          │
│          │  (MLP 256→64→1)  │                          │
│          └────────┬─────────┘                          │
│                   ▼                                    │
│            ┌────────────┐                              │
│            │ DC Power W │                              │
│            └────────────┘                              │
│                                                        │
└────────────────────────────────────────────────────────┘
""")

## 9️⃣ Launch Interactive Dashboard (Optional)

Run the cell below to launch the full Streamlit dashboard:

In [ ]:
# Launch Streamlit dashboard
# Note: This will start a background process

import subprocess
import time

# Start Streamlit in background
process = subprocess.Popen(
    ['streamlit', 'run', 'dashboard/app.py', '--server.headless=true'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(3)

# For Colab, get the public URL
try:
    from google.colab import output
    output.serve_kernel_port_as_window(8501)
    print("🌐 Dashboard launched! Click the link above to open.")
except:
    print("🌐 Dashboard running at: http://localhost:8501")
    print("   Open this URL in your browser.")

---

## 🎉 You've Completed the Demo!

### What You Learned:
1. ✅ How to create a multi-modal deep learning model
2. ✅ Cloud feature extraction from sky images
3. ✅ Sun position calculation using astronomical algorithms
4. ✅ Running power predictions with the model

### Next Steps:
- 📚 Read the full documentation on [GitHub](https://github.com/Pradsey5010/PV-Power-Estimation)
- 🔧 Train the model on real solar power data
- 🌐 Deploy for real-time predictions

### Contact:
- 📧 Email: pradyumnamand@gmail.com
- 📱 Phone: +1 480-797-3843